# Using Templates

Using [Jinja2](https://jinja.palletsprojects.com/en/3.0.x/) as template system. 

In [2]:
import jinja2 as J2
environment = J2.Environment()
template = environment.from_string("Hi {{ name }}, how are you?")
template.render(name="John")

'Hi John, how are you?'

A template system lets you replace a varible in a text. In fact, you can replace as many as variables as you want in all kinds of text. In all cases, the procedure is the same:

1. Create a template file. We will mark the file as template by using the extension `.j2`.
1. Get a list of values that you will use as input.
1. Write a script that will tie everything together.

For example, imagine you have a list of `hostname` and `IP` addresses, and want to create a configuration file for each host. The list of hosts is (see `hosts.txt`):

```
atta, 192.168.56.10
flik, 192.168.56.11
hopper, 192.168.56.12
```

And for each host create a configuration file for the Icinga monitoring system